In [82]:
!pip install apyori

In [83]:
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from apyori import apriori

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [84]:
df = pd.read_csv("/content/drive/My Drive/OnlineRetail.csv", encoding= 'unicode_escape')
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [85]:
df['Description'] = df['Description'].str.strip()
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df = df[~df['InvoiceNo'].str.contains('C')]

In [86]:
group = df.groupby(['InvoiceNo', 'InvoiceDate'])
group.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541894,581587,22631,CIRCUS PARADE LUNCH BOX,12,12/9/2011 12:50,1.95,12680.0,France
541895,581587,22556,PLASTERS IN TIN CIRCUS PARADE,12,12/9/2011 12:50,1.65,12680.0,France
541896,581587,22555,PLASTERS IN TIN STRONGMAN,12,12/9/2011 12:50,1.65,12680.0,France
541897,581587,22728,ALARM CLOCK BAKELIKE PINK,4,12/9/2011 12:50,3.75,12680.0,France


In [87]:
transactions = []

one = 0
two = 0
three = 0 
eight = 0

lcount = 0
linvoice = 0
longest = []

for i,j in group:
    if(lcount<len(j)):
        lcount = len(j)
        linvoice = i
        longest = list(j['Description'].map(str))
    
    if(len(j)==1):
        one+=1
    elif(len(j)==2):
        two+=1
    elif(len(j)==3):
        three+=1
    elif(len(j)==8):
        eight+=1
        
    transactions.append(list(j['Description'].map(str)))

In [88]:
print("There are ", one, "transactions with one item.")
print("There are ", two, "transactions with two items.")
print("There are ", three, "transactions with three items.")
print("There are ", eight, "transactions with eight items.")

There are  3718 transactions with one item.
There are  843 transactions with two items.
There are  697 transactions with three items.
There are  605 transactions with eight items.


In [89]:
print("Longest transaction :- ")
print ("Invoice number:", linvoice[0])
print ("Date: ", linvoice[1])
print ("Number of items: ", lcount)

Longest transaction :- 
Invoice number: 573585
Date:  10/31/2011 14:41
Number of items:  1114


In [90]:
df2 = pd.DataFrame(transactions)
df2.to_csv('transaction.csv')
df2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1074,1075,1076,1077,1078,1079,1080,1081,1082,1083,1084,1085,1086,1087,1088,1089,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099,1100,1101,1102,1103,1104,1105,1106,1107,1108,1109,1110,1111,1112,1113
0,WHITE HANGING HEART T-LIGHT HOLDER,WHITE METAL LANTERN,CREAM CUPID HEARTS COAT HANGER,KNITTED UNION FLAG HOT WATER BOTTLE,RED WOOLLY HOTTIE WHITE HEART.,SET 7 BABUSHKA NESTING BOXES,GLASS STAR FROSTED T-LIGHT HOLDER,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,HAND WARMER UNION JACK,HAND WARMER RED POLKA DOT,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,ASSORTED COLOUR BIRD ORNAMENT,POPPY'S PLAYHOUSE BEDROOM,POPPY'S PLAYHOUSE KITCHEN,FELTCRAFT PRINCESS CHARLOTTE DOLL,IVORY KNITTED MUG COSY,BOX OF 6 ASSORTED COLOUR TEASPOONS,BOX OF VINTAGE JIGSAW BLOCKS,BOX OF VINTAGE ALPHABET BLOCKS,HOME BUILDING BLOCK WORD,LOVE BUILDING BLOCK WORD,RECIPE BOX WITH METAL HEART,DOORMAT NEW ENGLAND,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,JAM MAKING SET WITH JARS,RED COAT RACK PARIS FASHION,YELLOW COAT RACK PARIS FASHION,BLUE COAT RACK PARIS FASHION,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,BATH BUILDING BLOCK WORD,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [91]:
print("Number of transactions : ", len(df2))
print ("NNumber of itmes: ", len(df))

Number of transactions :  22106
NNumber of itmes:  532621


In [92]:
rules = apriori(transactions , min_support = 0.01, min_confidence = 0.8, max_length = 10)

In [93]:
result = list(rules)
result

[RelationRecord(items=frozenset({'GREEN REGENCY TEACUP AND SAUCER', 'PINK REGENCY TEACUP AND SAUCER'}), support=0.02863475979372116, ordered_statistics=[OrderedStatistic(items_base=frozenset({'PINK REGENCY TEACUP AND SAUCER'}), items_add=frozenset({'GREEN REGENCY TEACUP AND SAUCER'}), confidence=0.8263707571801566, lift=17.99778518051679)]),
 RelationRecord(items=frozenset({'REGENCY TEA PLATE GREEN', 'REGENCY TEA PLATE PINK'}), support=0.012575771283814348, ordered_statistics=[OrderedStatistic(items_base=frozenset({'REGENCY TEA PLATE PINK'}), items_add=frozenset({'REGENCY TEA PLATE GREEN'}), confidence=0.9114754098360655, lift=52.88471236177444)]),
 RelationRecord(items=frozenset({'REGENCY TEA PLATE GREEN', 'REGENCY TEA PLATE ROSES'}), support=0.014430471365240206, ordered_statistics=[OrderedStatistic(items_base=frozenset({'REGENCY TEA PLATE GREEN'}), items_add=frozenset({'REGENCY TEA PLATE ROSES'}), confidence=0.837270341207349, lift=41.592580140965524)]),
 RelationRecord(items=frozen

In [94]:
print("Number of rules derived : ", len(result))

Number of rules derived :  39


In [95]:
for i in  range(10):
  print (i+1, ".")
  item = result[i] 
  pair=item[0]
  #print(pair)
  items=[x for x in pair]
  #print (items)
  print(items[0]+'->'+items[1])
  # second index of the inner list
  print("Support: " + str(item[1]))
  # third index of the list located at 0th
  # of the third index of the inner list

  print("Confidence: " + str(item[2][0][2]))
  print("Lift: " + str(item[2][0][3]))

1 .
GREEN REGENCY TEACUP AND SAUCER->PINK REGENCY TEACUP AND SAUCER
Support: 0.02863475979372116
Confidence: 0.8263707571801566
Lift: 17.99778518051679
2 .
REGENCY TEA PLATE GREEN->REGENCY TEA PLATE PINK
Support: 0.012575771283814348
Confidence: 0.9114754098360655
Lift: 52.88471236177444
3 .
REGENCY TEA PLATE GREEN->REGENCY TEA PLATE ROSES
Support: 0.014430471365240206
Confidence: 0.837270341207349
Lift: 41.592580140965524
4 .
REGENCY TEA PLATE PINK->REGENCY TEA PLATE ROSES
Support: 0.012168641997647698
Confidence: 0.8819672131147541
Lift: 43.81296002947136
5 .
SET/6 RED SPOTTY PAPER CUPS->SET/6 RED SPOTTY PAPER PLATES
Support: 0.015832805573147563
Confidence: 0.8177570093457944
Lift: 34.23737963749646
6 .
WOODEN TREE CHRISTMAS SCANDINAVIAN->WOODEN STAR CHRISTMAS SCANDINAVIAN
Support: 0.0127114810458699
Confidence: 0.8313609467455622
Lift: 35.89465837647929
7 .
ALARM CLOCK BAKELIKE CHOCOLATE->ALARM CLOCK BAKELIKE GREEN
Support: 0.01017823215416629
Confidence: 0.8122743682310469
Lift: 1

In [96]:
rules = apriori( transactions ,  min_confidence = 1)
result = list(rules)
print("Number of rules with 100% confidence : ", len(result))

Number of rules with 100% confidence :  0


In [97]:
rules = apriori( transactions ,  max_length = 3)
result = list(rules)
print("Number of rules with max_length(3) : ", len(result))
for i in range (len(result)):
  print (result)

Number of rules with max_length(3) :  1
[RelationRecord(items=frozenset({'WHITE HANGING HEART T-LIGHT HOLDER'}), support=0.1022346874151814, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'WHITE HANGING HEART T-LIGHT HOLDER'}), confidence=0.1022346874151814, lift=1.0)])]


In [98]:
def inspect(results):
    lhs = [tuple(result[2][0][0])[0] for result in results]
    rhs = [tuple(result[2][0][1])[0] for result in results]
    supports = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts = [result[2][0][3] for result in results]
    return list(zip(lhs, rhs, supports, confidences, lifts))

In [99]:
rules = apriori( transactions , min_support = 0.01, min_confidence = 0.8, max_length = 10)
result = list(rules)

In [100]:
resultsinDataFrame = pd.DataFrame(inspect(result), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])
resultsinDataFrame

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
0,PINK REGENCY TEACUP AND SAUCER,GREEN REGENCY TEACUP AND SAUCER,0.028635,0.826371,17.997785
1,REGENCY TEA PLATE PINK,REGENCY TEA PLATE GREEN,0.012576,0.911475,52.884712
2,REGENCY TEA PLATE GREEN,REGENCY TEA PLATE ROSES,0.014430,0.837270,41.592580
3,REGENCY TEA PLATE PINK,REGENCY TEA PLATE ROSES,0.012169,0.881967,43.812960
4,SET/6 RED SPOTTY PAPER CUPS,SET/6 RED SPOTTY PAPER PLATES,0.015833,0.817757,34.237380
5,WOODEN TREE CHRISTMAS SCANDINAVIAN,WOODEN STAR CHRISTMAS SCANDINAVIAN,0.012711,0.831361,35.894658
6,ALARM CLOCK BAKELIKE CHOCOLATE,ALARM CLOCK BAKELIKE RED,0.010178,0.812274,17.084812
7,LUNCH BAG BLACK SKULL.,RED RETROSPOT CHARLOTTE BAG,0.010133,0.829630,17.736743
8,STRAWBERRY CHARLOTTE BAG,RED RETROSPOT CHARLOTTE BAG,0.014657,0.859416,18.373559
9,WOODLAND CHARLOTTE BAG,RED RETROSPOT CHARLOTTE BAG,0.014611,0.821883,17.571126


In [101]:
resultsinDataFrame[resultsinDataFrame['Right Hand Side'].str.contains('METAL')]

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
